In [1]:
import numpy as np
from scipy import constants
from MeshedFields import *

In [11]:
filename="MeshGaussian.h5"
mf = MeshedField.ReadMeshedField(filename)

area = mf.MeshArea()
print("mesh area = %.3f cm²" % (1e4*np.sum(area)))

P = mf.EnergyFlowDensity()
print(P.shape)
Pz = P[:,2]
print("peak energy density = %.6f J/m²" % (np.max(Pz)))
print("total pulse energy = %.1f µJ" % (1e6*np.dot(area,Pz)))

mesh area = 50.185 cm²
(848, 3)
peak energy density = 3.711593 J/m²
total pulse energy = 2364.6 µJ


In [10]:
filename="NearTarget.h5"
target = MeshedField.ReadMeshedField(filename)

area = target.MeshArea()
print("mesh area = %.3f cm²" % (1e4*np.sum(area)))

P = target.EnergyFlowDensity()
print(P.shape)
Pz = P[:,2]
print("peak energy density = %.6f J/m²" % (np.max(Pz)))
print("total pulse energy = %.1f µJ" % (1e6*np.dot(area,Pz)))

target.ShowMeshedField(scalars=Pz,scalarTitle="Pz",showCenters=False)

mesh area = 50.185 cm²
(848, 3)
peak energy density = 3.555892 J/m²
total pulse energy = 2451.2 µJ


In [12]:
filename="Target.h5"
target = MeshedField.ReadMeshedField(filename)

area = target.MeshArea()
print("mesh area = %.3f cm²" % (1e4*np.sum(area)))

P = target.EnergyFlowDensity()
print(P.shape)
Pz = P[:,2]
print("peak energy density = %.6f J/m²" % (np.max(Pz)))
print("total pulse energy = %.1f µJ" % (1e6*np.dot(area,Pz)))

target.ShowMeshedField(scalars=Pz,scalarTitle="Pz",showCenters=False)

mesh area = 50.185 cm²
(848, 3)
peak energy density = 1.864030 J/m²
total pulse energy = 2273.2 µJ


In [4]:
mf.ShowMeshedField(scalars=Pz,scalarTitle="Pz",showCenters=False)

In [6]:
# this forces a separate plot window
%matplotlib qt

# plot a field trace when a cell is clicked
# subsequent plots are overlaid in the same window
# the plot window only becomes active when the VTK window is closed (it should be spawned as a separate thread)
def action(id):
    mf.ShowFieldTrace(id)

mf.ShowMeshedField(scalars=Pz,scalarTitle="Pz",showCenters=False, pickAction=action)

Compute local coordinate systems for all cells.<br>
The direction vectors (xi, eta, n) form a right-handed cartesic system.<br>
If the average normal vector is sufficiently far off the y direction xi is confined to lay in the x-z-plane.<br>
Otherwise eta is confined to lay in the y-z-plane.

In [30]:
normals = mf.MeshNormals()
avg_normal = np.average(normals,axis=0)
xi=[]
eta=[]
if (avg_normal[1] < 0.7):
    ey = np.array([0.0,1.0,0.0])
    for n in normals:
        x = np.cross(ey,n);
        xi.append(x/np.linalg.norm(x))
        y = np.cross(n,x);
        eta.append(y/np.linalg.norm(y))
else:
    ex = np.array([1.0,0.0,0.0])
    for n in normals:
        y = np.cross(n,ex);
        eta.append(y/np.linalg.norm(y))
        x = np.cross(y,n);
        xi.append(x/np.linalg.norm(x))

print(normals[1190])
print(xi[1190])
print(eta[1190])

[ 0.  0. -1.]
[-1.  0.  0.]
[ 0.  1.  0.]


find the neighbouring cell<br>
(all that share at least one point with the given cell)<br>
<br>
give the cell coordinates as local coordinates of the index cell

In [46]:
def Neighbourhood(self,index):
    ti = self.triangles[index]
    nbh = [n for n,t in enumerate(self.triangles) if (ti[0] in t) or (ti[1] in t) or (ti[2] in t)]
    return nbh

index = 1190
loc_coord = []
for cell in Neighbourhood(mf,index):
    d = mf.pos[cell]-mf.pos[index]
    x = np.dot(xi[cell],d)
    y = np.dot(eta[cell],d)
    z = np.dot(normals[cell],d)
    co = np.array([x,y,z])
    loc_coord.append(co)
    print(cell,co)
loc_coord = np.array(loc_coord)

1104 [-0.00099481 -0.00044175  0.        ]
1140 [ 0.00042228 -0.00046565  0.        ]
1183 [-0.00020744 -0.00143758  0.        ]
1190 [ 0.  0.  0.]
1270 [ -5.57469494e-04  -2.60680678e-05   0.00000000e+00]
1499 [ 0.00041984 -0.0011012   0.        ]
2703 [ 0.00083655  0.00077909  0.        ]
3898 [-0.00093923 -0.00107254  0.        ]
4223 [-0.0007036   0.00111343  0.        ]
4244 [-0.00015936  0.00122485  0.        ]
4247 [ 0.0011706   0.00032011  0.        ]
8412 [ 0.00022939  0.00068288  0.        ]
8669 [-0.00087231  0.00054539  0.        ]
8911 [ 0.00098572 -0.00024174  0.        ]


In [88]:
nbsize = [len(Neighbourhood(mf,index)) for index in range(mf.Np)]

In [90]:
np.histogram(nbsize, bins=range(22))

(array([   0,    0,    0,    0,    0,    0,    0,   37,   98,  111,  119,
         268, 1668, 3324, 2674,  936,  127,    4,    0,    0,    0]),
 array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16,
        17, 18, 19, 20, 21]))

compute the fields at a given time step over the cells of a neighbourhood

In [59]:
timestep = 200
Ex = np.array([mf.A[cell][timestep][0] for cell in Neighbourhood(mf,index)])
print(Ex)

[ 19880596.17819386  19985239.11522324  19863375.97145006
  19988966.68756156  19947417.72080901  19930709.87493346
  19968368.81161305  19836296.42043937  19878463.2733149   19914809.54206175
  19972052.19175592  19978866.39866267  19900012.66709729
  19981322.88169778]


show the field as a quadratic form over the transverse local coordinates (xi,eta)<br>
E = e00 + e10 xi + e01 eta + e20 xi² + e02 eta² + e11 xi eta

In [79]:
nb_xi = loc_coord[:,0]
nb_eta = loc_coord[:,1]
nb_xi2 = np.square(nb_xi)
nb_eta2 = np.square(nb_eta)
nb_xieta = nb_xi * nb_eta
nb_one = np.ones_like(nb_xi)

A = np.array([nb_one,nb_xi,nb_eta,nb_xi2,nb_eta2,nb_xieta]).transpose()
b = Ex
print(A.shape,A.dtype)
print(b.shape,b.dtype)
# solve the equation A x = b in a least-squares sense
# return x, chi², rank, singular values
x = np.linalg.lstsq(A, b)[0]
print(x)
print()

Ex_fit = np.dot(A,x)
print(Ex-Ex_fit)
print(np.dot(Ex-Ex_fit,Ex-Ex_fit))

(14, 6) float64
(14,) float64
[  1.99889249e+07   4.61787860e+07   7.57541788e+06  -4.98133825e+10
  -4.98126557e+10   5.25029951e+07]

[-47.60452619  35.45383242  -6.85187013  41.79889797 -52.72370413
 -51.57162689 -27.90055509  60.51893878  50.19586331 -27.31171433
 -10.25546595  17.32415453 -15.62399233  34.55176807]
20306.4549037


In [91]:
print(nb_xi)

[-0.00099481  0.00042228 -0.00020744  0.         -0.00055747  0.00041984
  0.00083655 -0.00093923 -0.0007036  -0.00015936  0.0011706   0.00022939
 -0.00087231  0.00098572]


In [93]:
print(nb_eta)

[ -4.41754205e-04  -4.65653171e-04  -1.43758046e-03   0.00000000e+00
  -2.60680678e-05  -1.10119617e-03   7.79085304e-04  -1.07253615e-03
   1.11342922e-03   1.22485325e-03   3.20114982e-04   6.82881840e-04
   5.45389748e-04  -2.41741653e-04]


### plot the field traces of a given cell

In [14]:
import matplotlib.pyplot as plt

In [17]:
trace = mf.A[1190]
NOTS = trace.shape[0]
print(NOTS)
t0 = mf.t0[1190]
dt = mf.dt
t = np.linspace(t0,t0+(NOTS-1)*dt, NOTS)

Ex = trace[:,0]
Ey = trace[:,1]
Ez = trace[:,2]
Bx = trace[:,3]
By = trace[:,4]
Bz = trace[:,5]

400


In [16]:
# this forces a separate plot window
%matplotlib qt

# prepare the plot geometry
left, width = 0.15, 0.80
rect1 = [left, 0.55, width, 0.40]  #left, bottom, width, height
rect2 = [left, 0.08, width, 0.40]
fig = plt.figure(1,figsize=(8,6))

ax1 = fig.add_axes(rect1)
ax4 = fig.add_axes(rect2, sharex=ax1)

# plot the time-trace of the fields

l1 = ax1.plot(t, Ex, "r-", label=r'$E_x$')
l2 = ax1.plot(t, Ey, "b-", label=r'$E_y$')
l3 = ax1.plot(t, Ez, "g-", label=r'$E_z$')

ax1.set_ylabel(r'$E$ [V/m]')
lines = l1 + l2 + l3
labels = [l.get_label() for l in lines]
ax1.legend(lines,labels,loc='upper right')
for label in ax1.get_xticklabels():
    label.set_visible(False)
ax1.grid(True)

l4 = ax4.plot(t, Bx, "r-", label=r'$B_x$')
l5 = ax4.plot(t, By, "b-", label=r'$B_y$')
l6 = ax4.plot(t, Bz, "g-", label=r'$B_z$')

ax4.set_ylabel(r'$B$ [T]')
ax4.set_xlabel(r't [ns]')
lines = l4 + l5 +l6
labels = [l.get_label() for l in lines]
ax4.legend(lines,labels,loc='upper right')
ax4.grid(True)

plt.show()